# Path
1. Create the model in GCP
    - Create the Dialogflow structure
    - Implement the PaLM into the Dialogflow
    - Test it out 
2. Make the backend locally
3. Create a Front End
4. Deploy the Front End

## Future Improvements
- Add identification in the frontend and change the conversational model if the user is identified. For example in the welcome intent instead of saying the fir interaction, get it ready to do something

---

# Imports
Importing all the libraries needed to run the notebook

In [46]:
from google.cloud import dialogflow
import uuid
from google.oauth2 import service_account
import os
from dotenv import load_dotenv
import gradio as gr
import random
import time
from PIL import Image
import numpy as np

Traceback (most recent call last):
  File "c:\Users\borga\Desktop\IE Computer Science\Semester 3\NLP Conversational Model\Final Project\myenv\lib\site-packages\gradio\routes.py", line 437, in run_predict
    output = await app.get_blocks().process_api(
  File "c:\Users\borga\Desktop\IE Computer Science\Semester 3\NLP Conversational Model\Final Project\myenv\lib\site-packages\gradio\blocks.py", line 1352, in process_api
    result = await self.call_function(
  File "c:\Users\borga\Desktop\IE Computer Science\Semester 3\NLP Conversational Model\Final Project\myenv\lib\site-packages\gradio\blocks.py", line 1077, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "c:\Users\borga\Desktop\IE Computer Science\Semester 3\NLP Conversational Model\Final Project\myenv\lib\site-packages\anyio\to_thread.py", line 33, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "c:\Users\borga\Desktop\IE Computer Science\Semester 3\NLP Conversational Model\Fina

In [20]:
# Load the environment variables
load_dotenv()

True

# Agent creation
---
**ADD IMAGES ONCE FINISHED**


# Agent connection
---
In order to connect to the project created in GCP and tHe agent created in Dialogflow we will have to follow a series of steps starting from authentication
### Authentication
Authentication as a service account in the Google Cloud Platform to access the resources in the cloud

In [21]:
# Authenticate the computer as a service account
service_account_key = os.getenv('service_account_key')
credentials = service_account.Credentials.from_service_account_file(service_account_key)

### Session Creation
A session represents a conversation between a Dialogflow agent and an end-user. You create a session at the beginning of a conversation and use it for each turn of the conversation. Once the conversation has ended, you discontinue using the session

In [22]:
session_id = uuid.uuid4().hex

In [23]:
# Load the variables to access the databases
project_id = os.getenv('project_id')
language_code = 'en'

In [24]:
# Create the session with the appropriate credentials
session_client = dialogflow.SessionsClient(credentials=credentials)

session = session_client.session_path(project_id, session_id)

# LLM

In [25]:
from vertexai.preview.language_models import TextGenerationModel
from google.cloud import aiplatform

In [26]:
region = os.getenv('region')
aiplatform.init(project=project_id, location=region)
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = service_account_key

In [27]:
# Test of text model
parameters = {
    "temperature": 0.2,  # Temperature controls the degree of randomness in token selection.
    "max_output_tokens": 1024,  # Token limit determines the maximum amount of text output.
    "top_p": 0.8,  # Tokens are selected from most probable to least until the sum of their probabilities equals the top_p value.
    "top_k": 40,  # A top_k of 1 means the selected token is the most probable among all tokens.
}

model = TextGenerationModel.from_pretrained("text-bison@001")
# response = model.predict('What are nutrients?', **parameters)
# print(f"Response from Model: {response.text}")

### Agent function
Function that given an input returns the output of by connecting to the agent

In [28]:
# Request an answer to our dialogflow agent according to a text
def dialogflow_request(text, language_code):
    text_input = dialogflow.TextInput(text=text, language_code=language_code)
    query_input = dialogflow.QueryInput(text=text_input)
    response = session_client.detect_intent(
        request={"session": session, "query_input": query_input}
    )
    response_message = str(response.query_result.fulfillment_messages[0].text.text[0])
    if "<<<llmquestion" in response_message:
        llm_response = model.predict((response_message.split("<<<llmquestion:")[1][:-3]), **parameters)
        return llm_response.text
    else:
        return response_message
    
    

# Roboflow Model

In [29]:
# Imports
from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

from ultralytics import YOLO
from IPython.display import display, Image

from roboflow import Roboflow

Ultralytics YOLOv8.0.20  Python-3.10.9 torch-2.0.1+cpu CPU
Setup complete  (8 CPUs, 7.8 GB RAM, 310.3/475.7 GB disk)


In [32]:
# Get the API key 
api_key = os.getenv('roboflow_api_key')

In [42]:
# Set up the model
rf = Roboflow(api_key=api_key)
project = rf.workspace().project("nutrition-object-detection")
model = project.version(1).model

results = model.predict("tryout.jpeg", confidence=50, overlap=50).json()

loading Roboflow workspace...
loading Roboflow project...
False


In [ ]:
def image_recognition(imagePath):
    # infer on a local image
    results = model.predict(imagePath, confidence=50, overlap=50).json()
    if results['predictions'] == []:
        prediction = 'Not found'
    else:
        prediction = results['predictions'][0]['class']

# Web Application 
---
Web application using Gradio to interact with the agent

### Handle Images
The user will hav the opportunity of inputting images to the chatbot. In order to do so the following functions are needed

In [48]:
# Function to get an answer from Dialogflow according to an input
def respond(message, chat_history):
    # Use the dialogflow function to get the answer
    bot_message = dialogflow_request(message, 'en')
    chat_history.append((message, bot_message))
    time.sleep(1)
    return "", chat_history

# Function to add text to the chat
def add_text(history, text):
    history.append((text, None))
    return history, respond(text)

# Function to add a file
def add_file(history, file):
    history.append(((file.name,), None))
    return history

# Gradio Web App

In [49]:
with gr.Blocks(css=".gradio-container {background-color: DarkSlateBlue}") as demo:
    disclaimer_agreed = gr.State(False)
    def conditions_met(newValue):
        disclaimer_agreed.value = newValue

    gr.Markdown("""
            <h1 style="text-align: center; color:white;" >FitBot</h1>
            <h4 style="text-align: center; color:white">Your personal food and health assistant</h4>
            """)

    with gr.Row():
        with gr.Column(scale=4):
            # Create a chatbot component and a model component
            chatbot = gr.Chatbot([], elem_id="chatbot", label="FitBot").style()
        with gr.Column(scale=1):
            gr.Markdown("""
            <h2 style="color:white" >Disclaimer</h2>
            <p style="color:white">The following bot is not meant following any regulatory health practice. All the information that is presented to you
            is generated from Vertex AI, the generative AI developed by Google. If you are interested in knowing more about the project and how this model
            was built, please refer to the following <a href="https://github.com/Niccoborg22/virtual-nutritionist-bot" style="color:white">Github link</a></p>
            """)

    with gr.Row():
        # Create a textbox component
        with gr.Column(scale=8):
            msg = gr.Textbox(
                show_label=False,
                placeholder="Enter text and press enter, or upload an image",
            ).style(container=False)
        with gr.Column(scale=1, min_width=0):
            # Create an upload button component to add images
            btn = gr.UploadButton("📁", file_types=["image"])

    with gr.Row(scale=0.7):
        clear_btn = gr.Button(value="🗑️  Clear")

    
    msg.submit(respond, [msg, chatbot], [msg, chatbot])
        
    # Create a message component for file uploads
    file_msg = btn.upload(add_file, [chatbot, btn], [chatbot], queue=False)
    clear_btn.click(lambda: None, None, chatbot, queue=False)

# launch the application
demo.launch()

The `style` method is deprecated. Please set these arguments in the constructor instead.
The `style` method is deprecated. Please set these arguments in the constructor instead.
You have unused kwarg parameters in Row, please remove them: {'scale': 0.7}


Running on local URL:  http://127.0.0.1:7869

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "c:\Users\borga\Desktop\IE Computer Science\Semester 3\NLP Conversational Model\Final Project\myenv\lib\site-packages\gradio\routes.py", line 437, in run_predict
    output = await app.get_blocks().process_api(
  File "c:\Users\borga\Desktop\IE Computer Science\Semester 3\NLP Conversational Model\Final Project\myenv\lib\site-packages\gradio\blocks.py", line 1352, in process_api
    result = await self.call_function(
  File "c:\Users\borga\Desktop\IE Computer Science\Semester 3\NLP Conversational Model\Final Project\myenv\lib\site-packages\gradio\blocks.py", line 1077, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "c:\Users\borga\Desktop\IE Computer Science\Semester 3\NLP Conversational Model\Final Project\myenv\lib\site-packages\anyio\to_thread.py", line 33, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "c:\Users\borga\Desktop\IE Computer Science\Semester 3\NLP Conversational Model\Fina